In [1]:
import openrouteservice
import folium
from folium.plugins import BeautifyIcon
import pandas as pd
import openrouteservice as ors
import pandas                  as pd
import numpy                   as np
import matplotlib.pyplot       as plt
import seaborn                 as sns
from sklearn.cluster import KMeans
from sklearn.cluster import OPTICS
from sklearn.cluster import SpectralClustering
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture
from sklearn.cluster import AffinityPropagation
import warnings
warnings.filterwarnings("ignore")

In [15]:
df=pd.DataFrame(pd.read_csv('coords.csv'))
df

,Lat,Lon,Volumen
0,25.660137,-100.275035,1
1,25.659375,-100.272048,1
2,25.659737,-100.271438,1
3,25.662988,-100.272437,1
4,25.667788,-100.275417,1
5,25.667920,-100.269800,1
6,25.667014,-100.276777,1
7,25.668238,-100.276907,1
8,25.644772,-100.301994,1
9,25.649527,-100.307434,1


In [22]:
def routesGenerator(model, n_clusters, mode):

    nk = n_clusters
    df_K = pd.DataFrame(df['Lat'])
    df_K['Lon'] = df['Lon']
    #df_K

    if (model == 1):
        kmeans = KMeans(n_clusters= nk).fit(df_K)
        df_K['Grupo'] = kmeans.predict(df_K)
    elif (model == 2):
        AgC = AgglomerativeClustering(n_clusters= nk).fit(df_K)
        df_K['Grupo'] = AgC.fit_predict(df_K)
    elif (model == 3):
        sc = SpectralClustering(n_clusters= nk).fit(df_K)
        df_K['Grupo'] = sc.fit_predict(df_K)
    elif (model == 4):
        GM = GaussianMixture(n_components= nk).fit(df_K)
        df_K['Grupo'] = GM.predict(df_K)
    #df_K['Needed_Amount'] = df['cantidad']
    #df_K['volumen'] = df['volxpostal']

    z = folium.Map(location=[25.65240416152182, -100.29108458215048], tiles='cartodbpositron', zoom_start=13)
    colors = ['green', 'red', 'blue', 'yellow', 'purple', 'brown', 'grey', 'pink']
    keyAPI = '5b3ce3597851110001cf624804463b8629234d55bcf497117d4c2c11'
    times = []
    distanceS = []


    #ors_client = 

    #if (1==1):
    #    i=0
    for i in range(nk):
        df_99=df_K[df_K['Grupo']==i]
        del df_99['Grupo']
        df_99['ID'] = list(range(len(df_99)))
        df_99["Open_From"] = (len(df_99)*"08:00 ").split()
        df_99["Open_To"] = (len(df_99)*"20:00 ").split()
        df_99['Needed_Amount'] = (len(df_99)*"1 ").split()
        df_99.to_csv('df_99.csv')

        

        # Next load the delivery locations from CSV file at ../resources/data/idai_health_sites.csv
        # ID, Lat, Lon, Open_From, Open_To, Needed_Amount
        deliveries_data = pd.read_csv(
            'df_99.csv',
            index_col="ID",
            parse_dates=["Open_From", "Open_To"]
        )

        for location in deliveries_data.itertuples():
            tooltip = folium.map.Tooltip("<h4><b>ID {}</b></p><p>Supplies needed: <b>{}</b></p>".format(
                location.Index, location.Needed_Amount
            ))

            folium.Marker(
                location=[location.Lat, location.Lon],
                tooltip=tooltip,
                icon=BeautifyIcon(
                    icon_shape='marker',
                    number=int(location.Index),
                    spin=True,
                    text_color=colors[i],
                    background_color="#FFF",
                    inner_icon_style="font-size:12px;padding-top:-5px;"
                )
            ).add_to(z)

            
        depot = [25.65240416152182, -100.29108458215048]

        folium.Marker(
            location=depot,
            icon=folium.Icon(color="green", icon="bus", prefix='fa'),
            setZIndexOffset=1000
        ).add_to(z)

        vehicles = list()
        for idx in range(3):
            vehicles.append(
                ors.optimization.Vehicle(
                    id=i,
                    start=list(reversed(depot)),
                    end=list(reversed(depot)),
                    capacity=[300000] 
                )
            )

        deliveries = list()
        for delivery in deliveries_data.itertuples():
            #if (delivery.volumen /delivery.Needed_Amount < 0.51):
                #print(delivery.volumen /delivery.Needed_Amount)
            #    serviceO=300*delivery.Needed_Amount
            #else:
                #print(delivery.volumen /delivery.Needed_Amount)
            #    serviceO=600*delivery.Needed_Amount
                
            deliveries.append(
                ors.optimization.Job(
                    id=delivery.Index,
                    location=[delivery.Lon, delivery.Lat],
                    service = 60,
                    amount=[delivery.Needed_Amount],
                )
            )

        # Get an API key from https://openrouteservice.org/dev/#/signup
        result = ors.Client(key= keyAPI).optimization(
            jobs=deliveries,
            vehicles=vehicles,
            geometry=True
        )

        # Add the output to the map
        for color, route in zip([colors[i],colors[i],colors[i]], result['routes']):
            decoded = ors.convert.decode_polyline(route['geometry'])  # Route geometry is encoded
            gj = folium.GeoJson(
                name='Vehicle {}'.format(route['vehicle']),
                data={"type": "FeatureCollection", "features": [{"type": "Feature",
                                                                "geometry": decoded,
                                                                "properties": {"color": color}
                                                                }]},
                style_function=lambda x: {"color": x['properties']['color']}
            )
            gj.add_child(folium.Tooltip(
                """<h4>Vehicle {vehicle}</h4>
                <b>Distance</b> {distance} m <br>
                <b>Duration</b> {duration} secs
                """.format(**route)
            ))
            gj.add_to(z)

        
        #time
        last_step=result['routes'][0]['steps'][-1]
        timeR = last_step['arrival']+last_step.get('service',0)
        #print(timeR/3600)
        times.append(timeR/3600)
        #print(pd.to_datetime(timeR, unit='s'))

        #distance
        last_step=result['routes'][0]['steps'][-1]
        distanceR = last_step['distance']
        #print(timeR/3600)
        distanceS.append(distanceR/1000)
        

    folium.LayerControl().add_to(z)
    totaltime = sum(times)
    totaldistance = sum(distanceS)
    maxtime = max(times)


    
    if (mode == 1):
        return(maxtime, totaldistance)
    elif(mode == 3):
        return (times,totaldistance)
    elif(mode == 2):
        return z
    elif (mode == 4):
        return (times,distanceS)
    elif (mode == 5):
        return result

In [16]:
def routesData(nMaxRutas):
    maxRutas = nMaxRutas
    dfporK = pd.DataFrame()
    dfporK['K'] = list(range(1,maxRutas))
    models = ['Kmeans_', 'AgC_', 'Spectre_', 'Gaussian_']

    #Models
    for j in range(4):
        timedf = []
        distancedf = []
        for i in range(1,maxRutas):
            timedf.append(routesGenerator(j+1,i,1)[0])
            distancedf.append(routesGenerator(j+1,i,1)[1])
        dfporK[models[j]+'Tiempo Max'] = timedf
        dfporK[models[j]+'Distancia total'] = distancedf
    
    return dfporK

In [17]:
dfrutasdata= routesData(3)

In [18]:
dfrutasdata

,K,Kmeans_Tiempo Max,Kmeans_Distancia total,AgC_Tiempo Max,AgC_Distancia total,Spectre_Tiempo Max,Spectre_Distancia total,Gaussian_Tiempo Max,Gaussian_Distancia total
0,1,10.990000,108.613,10.990000,108.613,10.990000,108.613,10.990000,108.613
1,2,6.849444,115.473,6.849444,115.473,6.849444,115.473,7.453056,136.601


In [19]:
sal = 177.525
costoKilometro = 22.97/9.57

def routesCosto(salario, costokilometro, dfroutesdata):
    salarioPorHora = salario
    costokilometro = costokilometro
    dfCopy = dfroutesdata.copy()
    dfCostos = pd.DataFrame()
    dfCostos['K'] = [1,2]

    for j in range(4):
        costo = dfCopy[dfCopy.columns[2*j+1]]*177.525*dfCopy[dfCopy.columns[0]]+dfCopy[dfCopy.columns[2*j+2]]*costokilometro
        dfCostos['Costo modelo ' + str(j+1)] = costo

    dfCostos.columns = ['K', 'Kmeans', 'Agglomerative cluster', 'Spectre cluster', 'Gaussian Mixture']    

    return dfCostos


dfporK = routesCosto(sal, costoKilometro, dfrutasdata)
dfporK

,K,Kmeans,Agglomerative cluster,Spectre cluster,Gaussian Mixture
0,1,2211.693649,2211.693649,2211.693649,2211.693649
1,2,2709.054582,2709.054582,2709.054582,2974.078323


In [23]:
routesGenerator(1,2,4)

([1.9494444444444445, 1.5052777777777777], [58.917, 56.556])

In [24]:
routesGenerator(1,2,2)